In [64]:
import nltk
import pprint
import gensim
from nltk.corpus import reuters
from gensim.models import Word2Vec

print("NLTK version: {}".format(nltk.__version__))
print("Regex version: {}".format(re.__version__))
print("Gensim version: {}".format(gensim.__version__))

NLTK version: 3.3
Regex version: 2.2.1
Gensim version: 3.8.0


# 1. Data Ingestion

In [41]:
def read_corpus(category="crude"):
    files = reuters.fileids(category)
    # We are extracting the words in lower case as it is a common data cleaning activity
    return [w.lower() for f in files for w in list(reuters.words(f))]

In [44]:
reuters_corpus = read_corpus()
pprint.pprint(reuters_corpus, compact=True, width=100)

['japan', 'to', 'revise', 'long', '-', 'term', 'energy', 'demand', 'downwards', 'the', 'ministry',
 'of', 'international', 'trade', 'and', 'industry', '(', 'miti', ')', 'will', 'revise', 'its',
 'long', '-', 'term', 'energy', 'supply', '/', 'demand', 'outlook', 'by', 'august', 'to', 'meet',
 'a', 'forecast', 'downtrend', 'in', 'japanese', 'energy', 'demand', ',', 'ministry', 'officials',
 'said', '.', 'miti', 'is', 'expected', 'to', 'lower', 'the', 'projection', 'for', 'primary',
 'energy', 'supplies', 'in', 'the', 'year', '2000', 'to', '550', 'mln', 'kilolitres', '(', 'kl',
 ')', 'from', '600', 'mln', ',', 'they', 'said', '.', 'the', 'decision', 'follows', 'the',
 'emergence', 'of', 'structural', 'changes', 'in', 'japanese', 'industry', 'following', 'the',
 'rise', 'in', 'the', 'value', 'of', 'the', 'yen', 'and', 'a', 'decline', 'in', 'domestic',
 'electric', 'power', 'demand', '.', 'miti', 'is', 'planning', 'to', 'work', 'out', 'a', 'revised',
 'energy', 'supply', '/', 'demand', 'out

 '248', '.', '1', 'mln', 'barrels', 'and', 'refinery', 'crude', 'oil', 'stocks', 'rose', '6', '.',
 '3', 'mln', 'barrels', 'to', '335', '.', '8', 'mln', '.', 'the', 'eia', 'said', 'residual', 'fuel',
 'stocks', 'fell', '100', ',', '000', 'barrels', 'to', '38', '.', '1', 'mln', 'barrels', 'and',
 'crude', 'oil', 'stocks', 'in', 'the', 'strategic', 'petroleum', 'reserve', '(', 'spr', ')',
 'rose', '1', '.', '1', 'mln', 'barrels', 'to', '520', '.', '0', 'mln', '.', 'the', 'total', 'of',
 'all', 'crude', ',', 'refined', 'product', 'and', 'spr', 'stocks', 'rose', '9', '.', '4', 'mln',
 'barrels', 'to', '1', ',', '561', '.', '1', ',', 'it', 'said', '.', 'recent', 'u', '.', 's', '.',
 'oil', 'demand', 'off', '2', '.', '6', 'pct', 'from', 'year', 'ago', 'u', '.', 's', '.', 'oil',
 'demand', 'as', 'measured', 'by', 'products', 'supplied', 'fell', '2', '.', '6', 'pct', 'in',
 'the', 'four', 'weeks', 'ended', 'april', 'three', 'to', '15', '.', '73', 'mln', 'barrels', 'per',
 'day', '(', 'bpd', ')

 'barrel', 'of', 'oil', 'will', 'fall', 'below', '18', ',', 'perhaps', '17', '.', '80', 'dlrs',
 'this', 'week', 'or', 'next', 'if', 'there', 'is', 'no', 'control', 'on', 'supplies', ',"', 'it',
 'said', '.', '"', 'the', 'sources', 'believed', 'a', 'return', 'of', 'oil', 'below', '18', 'dlrs',
 'a', 'barrel', 'may', 'open', 'the', 'doors', 'for', 'international', 'oil', 'companies', 'to',
 'pressure', 'opec', 'over', 'contract', 'prices', ',', 'similar', 'to', 'the', 'struggle', 'last',
 'march', ',"', 'it', 'said', ',', 'apparently', 'referring', 'to', 'resistance', 'by', 'buyers',
 'to', 'lift', 'from', 'qatar', 'unless', 'it', 'gave', 'price', 'discounts', '.', '"', 'more',
 'than', 'one', 'official', 'has', 'warned', 'opec', 'would', 'find', 'its', 'solidarity', 'under',
 'scrutiny', 'by', 'the', 'end', 'of', 'april', 'or', 'start', 'of', 'may', ',"', 'it', 'said', ',',
 'noting', 'demand', 'usually', 'fell', 'with', 'the', 'onset', 'of', 'summer', '.', 'texaco', '&',
 'lt', ';', '

 'no', 'great', 'change', ',"', 'said', 'another', 'trader', ',', 'adding', 'that', 'traders',
 'are', 'tending', 'toward', 'prudence', 'in', 'their', 'dealings', 'with', 'the', 'company', '.',
 'traders', 'are', 'assessing', 'whether', 'to', 'require', 'cash', 'prepayment', 'or', 'letters',
 'of', 'credit', ',', 'or', 'to', 'continue', 'to', 'trade', 'as', 'usual', 'with', 'texaco', 'on',
 'an', 'open', 'line', 'basis', ',', 'he', 'said', '.', 'another', 'trader', ',', 'however', ',',
 'described', 'today', "'", 's', 'activity', 'as', 'business', 'as', 'usual', ',', 'adding', 'that',
 'traders', 'feel', 'more', 'secure', 'because', 'no', 'liens', 'can', 'be', 'put', 'on', 'texaco',
 "'", 's', 'assets', 'while', 'it', 'is', 'in', 'bankruptcy', '.', 'traders', 'said', 'there',
 'was', 'no', 'apparent', 'effect', 'of', 'the', 'texaco', 'filing', 'on', 'crude', 'futures',
 'trading', 'although', 'they', 'said', 'the', 'exchange', 'might', 'lower', 'texaco', "'", 's',
 'position', 'limit',

 'gulf', 'situation', 'with', 'allied', 'leaders', 'at', 'next', 'week', "'", 's', 'venice',
 'economic', 'summit', 'and', 'hinted', 'he', 'would', 'seek', 'their', 'help', 'in', 'preserving',
 'free', 'navigation', '.', 'in', 'a', 'speech', 'prepared', 'for', 'delivery', 'as', 'the',
 'united', 'states', 'made', 'plans', 'to', 'protect', '11', 'kuwaiti', 'oil', 'tankers', 'from',
 'iranian', 'attack', ',', 'reagan', 'said', 'the', 'american', 'people', 'were', 'aware', 'that',
 '"', 'it', 'is', 'not', 'our', 'interests', 'alone', 'that', 'are', 'being', 'protected', '."',
 'saying', 'that', 'allied', 'dependence', 'on', 'gulf', 'oil', 'was', 'no', 'secret', ',',
 'reagan', 'declared', ',', '"', 'during', 'the', 'upcoming', 'summit', 'in', 'venice', ',', 'we',
 'will', 'be', 'discussing', 'the', 'common', 'security', 'interests', 'shared', 'by', 'the',
 'western', 'democracies', 'in', 'the', 'mideast', 'gulf', '.', '"', 'the', 'future', 'belongs',
 'to', 'the', 'brave', '.', 'free', 'm

...

 'has', 'announced', 'a', 'series', 'of', 'measures', 'since', 'may', 'intended', 'to', 'stimulate',
 'exports', ',', 'encourage', 'foreign', 'investment', 'and', 'revitalise', 'the', 'economy', '.',
 'however', ',', 'key', 'monopolies', 'in', 'areas', 'like', 'steel', 'and', 'plastics', 'and',
 'high', 'tariff', 'barriers', 'remain', 'in', 'place', '.', 'conable', 'arrived', 'in',
 'indonesia', 'on', 'saturday', 'and', 'has', 'since', 'met', '14', 'indonesian', 'cabinet',
 'ministers', 'to', 'review', 'the', 'country', "'", 's', 'borrowing', 'needs', 'and', 'the',
 'impact', 'of', 'falling', 'oil', 'prices', 'on', 'the', 'country', '.', 'kuwait', 'says', 'opec',
 '2', '.', '4', 'mln', 'bpd', 'below', 'ceiling', 'kuwaiti', 'oil', 'minister', 'sheikh', 'ali',
 'al', '-', 'khalifa', 'al', '-', 'sabah', 'said', 'opec', 'was', 'producing', 'well', 'below',
 'its', 'oil', 'output', 'ceiling', 'and', 'this', 'would', 'help', 'prices', 'move', 'higher', ',',
 'according', 'to', 'a', 'kuwaiti'

 'argued', 'that', 'the', 'drilling', 'would', 'hurt', 'native', 'hunting', 'and', 'fishing', '.',
 'lower', 'tax', 'offsets', 'lower', 'shell', 'u', '.', 'k', '.', 'upstream', 'profit', '&', 'lt',
 ';', 'shell', 'u', '.', 'k', '.', 'ltd', "'", 's', '>', 'pre', '-', 'tax', 'profit', 'on',
 'exploration', 'and', 'production', 'operations', 'fell', 'to', '869', 'mln', 'stg', 'in', '1986',
 'from', '2', '.', '12', 'billion', 'in', '1985', 'due', 'to', 'the', 'fall', 'in', 'oil', 'prices',
 'last', 'year', ',', 'shell', 'u', '.', 'k', '.', 'finance', 'director', 'nigel', 'haslam', 'said',
 '.', 'but', 'he', 'told', 'a', 'press', 'conference', 'that', 'due', 'to', 'the', 'high',
 'marginal', 'tax', 'rate', 'on', 'north', 'sea', 'operations', ',', 'the', 'main', 'impact', 'of',
 'the', 'drop', 'in', 'profit', 'was', 'absorbed', 'by', 'a', 'fall', 'in', 'taxation', 'to', '330',
 'mln', 'stg', 'from', '1', '.', '45', 'billion', 'in', '1985', '.', 'the', 'bulk', 'of', 'tax',
 'last', 'year', 'w

 'the', 'state', 'oil', 'company', 'will', 'manage', '.', 'pdvsa', 'officials', 'said', 'that',
 'among', 'those', 'who', 'bid', 'for', 'the', 'partnership', 'is', 'a', 'consortium', 'between',
 'agip', 'carbone', ',', 'a', 'subsidiary', 'of', 'italy', "'", 's', 'ente', 'nazionale',
 'idrocarburi', '(', 'eni', '),', 'and', 'atlantic', 'richfield', '&', 'lt', ';', 'arc', '>', 'of',
 'the', 'united', 'states', '.', 'minister', 'of', 'energy', 'and', 'mines', 'arturo', 'hernandez',
 'grisanti', 'said', 'discussions', 'are', 'currently', 'taking', 'place', 'to', 'finalize', 'the',
 'terms', 'of', 'the', 'contract', 'with', 'the', 'foreign', 'partner', '.', 'pdvsa', 'vice', '-',
 'president', 'pablo', 'reimpell', 'said', 'last', 'week', 'the', 'first', 'shipment', 'of', 'coal',
 'from', 'the', 'carbozulia', 'project', 'should', 'be', 'made', 'during', 'the', 'final',
 'quarter', 'of', '1987', ',', 'and', 'would', 'measure', 'between', '100', '-', '150', ',', '000',
 'metric', 'tons', '.', '

 'will', 'inject', 'about', '350', 'mln', 'dlrs', 'a', 'year', 'into', 'the', 'oil', 'and', 'gas',
 'industry', 'and', 'could', 'lead', 'to', 'more', 'than', 'one', 'billion', 'dlrs', 'in', 'new',
 'investment', ',', 'masse', 'told', 'a', 'news', 'conference', '.', 'the', 'program', 'will',
 'affect', 'drilling', 'done', 'anywhere', 'in', 'canada', 'on', 'or', 'after', 'april', '1', ',',
 '1987', '.', 'masse', 'told', 'reporters', 'that', 'the', 'government', "'", 's', 'oil',
 'industry', 'aid', 'package', 'is', 'aimed', 'at', 'small', 'and', 'medium', 'sized', 'companies',
 '.', 'the', 'aid', 'package', ',', 'called', 'the', 'canadian', 'exploration', 'and',
 'development', 'incentive', 'program', ',', 'will', 'restrict', 'the', 'total', 'payments', 'that',
 'any', 'individual', 'company', 'can', 'claim', 'to', '10', 'mln', 'dlrs', 'a', 'year', '.',
 'masse', 'said', 'the', 'program', 'will', 'probably', 'generate', 'new', 'employment',
 'equivalent', 'to', '20', ',', '000', 'people',

 'chinese', '-', 'built', 'missiles', 'to', 'cover', 'the', 'entrance', 'to', 'the', 'gulf', '.',
 'weinberger', 'told', 'reporters', 'prior', 'to', 'a', 'speech', 'at', 'texas', 'christian',
 'university', 'that', 'he', 'did', 'not', 'think', 'iran', 'and', 'the', 'united', 'states',
 'were', 'moving', 'towards', 'a', 'potential', 'conflict', ',', 'adding', 'that', 'the', 'straits',
 'of', 'hormuz', 'at', 'the', 'mouth', 'of', 'the', 'gulf', 'were', 'still', '"', 'free', 'water',
 '."', 'gulf', 'escorts', 'still', 'under', 'discussion', '-', 'weinberger', 'no', 'action', 'has',
 'been', 'taken', 'yet', 'on', 'the', 'reagan', 'adminstration', "'", 's', 'offer', 'to', 'escort',
 'kuwaiti', 'oil', 'tankers', 'through', 'the', 'gulf', ',', 'but', 'the', 'issue', 'is', 'being',
 'discussed', ',', 'u', '.', 's', '.', 'secretary', 'of', 'defence', 'caspar', 'weinberger', 'said',
 '.', 'the', 'offer', 'was', 'made', 'to', 'kuwait', 'in', 'light', 'of', 'iran', "'", 's',
 'deployment', 'of', '

In [46]:
reuters_string = ' '.join(reuters_corpus)
pprint.pprint(reuters_string, compact=True, width=100)

('japan to revise long - term energy demand downwards the ministry of international trade and '
 'industry ( miti ) will revise its long - term energy supply / demand outlook by august to meet a '
 'forecast downtrend in japanese energy demand , ministry officials said . miti is expected to '
 'lower the projection for primary energy supplies in the year 2000 to 550 mln kilolitres ( kl ) '
 'from 600 mln , they said . the decision follows the emergence of structural changes in japanese '
 'industry following the rise in the value of the yen and a decline in domestic electric power '
 'demand . miti is planning to work out a revised energy supply / demand outlook through '
 'deliberations of committee meetings of the agency of natural resources and energy , the '
 'officials said . they said miti will also review the breakdown of energy supply sources , '
 "including oil , nuclear , coal and natural gas . nuclear energy provided the bulk of japan ' s "
 'electric power in the fiscal yea

 'oil prices to gradually increase over the next two years as u . s . reliance on imports of oil '
 'from the middle east grows . " i believe we have bottomed out and can look forward to a trend of '
 'gradually increasing prices ," c . w . nance , president of tenneco oil exploration and '
 'production , told a meeting of the petroleum equipment suppliers association . nance predicted '
 'that by 1990 , the organization of producing and exporting countries would be producing at the '
 'rate of 80 pct of capacity . the gain will come largely through increased imports to the united '
 'states , he said . " they will be able to raise the price again but i do not think they will '
 'raise it as much as they did in 1979 ," nance said . he did not say how much of a price hike he '
 'expected . andrew shoup , chairman of dallas - based sabine , predicted that world oil prices '
 'would increase from a range of 15 to 20 dlrs a barrel in 1987 to a range of 17 to 22 dlrs a '
 'barrel in 1988 . 

 'oil analysts said if opec keeps group output close to its first - half 1987 ceiling of 15 . 8 '
 'mln bpd , supply and demand would be balanced by the end of the second quarter . they also said '
 'if opec holds its official price structure based on a reference price of 18 dlrs , spot price '
 'fluctuations should be limited to a 16 . 50 to 17 . 20 dlrs range for dubai and a 17 to 18 dlrs '
 'range for the north sea \' s brent blend . one japanese refiner said , " at the moment there \' '
 's a lot of pressure on opec , particularly on qatar . but if they hold out there will be no '
 'problem , and i \' m beginning to trust their ability ." nigerian oil minister and opec '
 'president rilwanu lukman told a news conference in lagos yesterday , " nigeria and all member '
 'countries of opec remain determined to uphold the december agreement by adhering strictly to '
 'their various quotas and official selling prices ." he said opec believed consumers had drawn '
 'heavily on stocks of 

 'had a very serious impact on venezuela ." pdvsa \' s export volume averaged 1 . 508 mln barrels '
 'a day , of which 658 , 000 bpd were crude oil and 850 , 000 bpd refined products . the figure '
 "surpassed pdvsa ' s stated goal of 1 . 410 mln bpd and the 1985 export volume of 1 . 371 mln bpd "
 '. but it was not enough to cover the losses from a drop in the average price from 25 . 88 dlrs '
 'per barrel in 1985 to 13 . 90 dlrs last year . the 13 . 90 per barrel price for 1986 was 1 . 01 '
 'dlrs higher than the figure originally announced by the central bank . hernandez said the '
 "increase had come from a new accounting system which included the results of pdvsa ' s overseas "
 "refining and marketing operations . consumption in venezuela ' s internal market increased from "
 '323 , 000 bpd in 1985 to 342 , 000 bpd last year . however , hernandez stressed that the '
 'government had managed to keep gasoline sales almost constant at 164 , 000 bpd . exploration by '
 'pdvsa led to 

# 3. Data Cleaning

In [54]:
# Removing digits and special characters
processed = re.sub('[^a-zA-Z]', ' ',reuters_string)
# Removing special characters
processed = re.sub(r'\s+', ' ', processed)

# Let's do word tokenization
all_words = nltk.word_tokenize(processed)

In [77]:
print("First 10 words in the corpus:\n {}".format(all_words))

First 10 words in the corpus:
 ['japan', 'to', 'revise', 'long', 'term', 'energy', 'demand', 'downwards', 'the', 'ministry', 'of', 'international', 'trade', 'and', 'industry', 'miti', 'will', 'revise', 'its', 'long', 'term', 'energy', 'supply', 'demand', 'outlook', 'by', 'august', 'to', 'meet', 'a', 'forecast', 'downtrend', 'in', 'japanese', 'energy', 'demand', 'ministry', 'officials', 'said', 'miti', 'is', 'expected', 'to', 'lower', 'the', 'projection', 'for', 'primary', 'energy', 'supplies', 'in', 'the', 'year', 'to', 'mln', 'kilolitres', 'kl', 'from', 'mln', 'they', 'said', 'the', 'decision', 'follows', 'the', 'emergence', 'of', 'structural', 'changes', 'in', 'japanese', 'industry', 'following', 'the', 'rise', 'in', 'the', 'value', 'of', 'the', 'yen', 'and', 'a', 'decline', 'in', 'domestic', 'electric', 'power', 'demand', 'miti', 'is', 'planning', 'to', 'work', 'out', 'a', 'revised', 'energy', 'supply', 'demand', 'outlook', 'through', 'deliberations', 'of', 'committee', 'meetings', 

# 3. Creating a Word2Vec Model

In [90]:
word2vec = Word2Vec([all_words],min_count=2)
word2vec.train([all_words],total_examples=len(all_words),epochs=20,)
# The min_count parameter specifies to include only those words in the Word2Vec model that appear at least twice in the corpus

(200000, 2356900)

In [91]:
# Let us see the dictionary of unquie words that exist at least twice in our corpus
vocabulary = word2vec.wv.vocab
print(vocabulary)

{'japan': <gensim.models.keyedvectors.Vocab object at 0x1a22d990b8>, 'to': <gensim.models.keyedvectors.Vocab object at 0x1a22d99390>, 'revise': <gensim.models.keyedvectors.Vocab object at 0x1a22d99a90>, 'long': <gensim.models.keyedvectors.Vocab object at 0x1a22d99668>, 'term': <gensim.models.keyedvectors.Vocab object at 0x1a22d99710>, 'energy': <gensim.models.keyedvectors.Vocab object at 0x1a22d996a0>, 'demand': <gensim.models.keyedvectors.Vocab object at 0x1a22d995f8>, 'downwards': <gensim.models.keyedvectors.Vocab object at 0x1a22d99550>, 'the': <gensim.models.keyedvectors.Vocab object at 0x1a22d99b38>, 'ministry': <gensim.models.keyedvectors.Vocab object at 0x1a22d99208>, 'of': <gensim.models.keyedvectors.Vocab object at 0x1a22d99160>, 'international': <gensim.models.keyedvectors.Vocab object at 0x1a22d99048>, 'trade': <gensim.models.keyedvectors.Vocab object at 0x1a22d99400>, 'and': <gensim.models.keyedvectors.Vocab object at 0x1a22d99320>, 'industry': <gensim.models.keyedvectors.V

# 4. Model Analysis

In [92]:
# Let's find a vector of a particular word
v1 = word2vec.wv['energy']
print(v1)
# By default a 100 dimensional vector is created by Gensim

[-0.07189274  0.03447092 -0.53912383  0.8040508   0.5845938  -0.8036414
  0.53862303 -0.16626348  0.94661057 -0.7201925   0.39015597 -0.6161888
  0.09925918  0.1303872   1.08934    -0.23541509 -0.89622563 -0.6661813
  0.29209623  0.5839784  -1.6094568   0.7321111   0.16841972 -0.08206341
  0.6592902   0.32737005 -0.29180855 -0.18636322 -0.13990806  0.6011794
 -0.09064224  0.5308381  -1.1433158   0.02005624 -0.03419577 -0.68414533
  0.32423332 -0.2808213  -0.07042054  0.60600567 -1.3304747  -0.7542913
 -1.6449203   0.9768572  -0.6977302  -0.25825188  0.6084306   0.91464484
  0.23796377  1.474066    1.1322174   0.10835785  0.3655007  -1.403337
 -0.6953216   0.07532396 -0.15340593 -0.19551507 -0.05087881  0.6166431
 -0.3601313   1.0881126  -0.18114582  0.3886247  -0.6450301  -0.68699706
 -1.4472305   0.9737025  -0.6203745  -0.83741     0.6012484  -0.60852885
  0.40085155  0.23549739 -0.23490563 -0.47821492 -1.2741116   1.1869137
  0.40821695 -0.27886653  1.6398345   0.34013906  1.34052   

In [95]:
# Let's print similar words
sim_words = word2vec.wv.most_similar('petrochemical')
print(sim_words)
# We can see similar words to 'energy' along with similarity index. Chemical is obviously very similar to petrochemical. Tanker is used often with petrochemical

[('chemical', 0.9998959302902222), ('major', 0.9998775720596313), ('their', 0.9998685717582703), ('tanker', 0.9998661279678345), ('industry', 0.9998646974563599), ('with', 0.9998601675033569), ('foreign', 0.9998576045036316), ('companies', 0.9998564720153809), ('yesterday', 0.9998536109924316), ('chairman', 0.9998525381088257)]
